In [3]:
import json
import cv2
import numpy as np
from mtcnn import MTCNN
from scipy.spatial.distance import cosine
from tensorflow.keras.models import load_model

In [4]:
# Inicializar os modelos
detector = MTCNN()   # Para detectar rostos
embedder_path = 'facenet_keras.h5'
embedder = load_model(embedder_path)

In [5]:
# Carregar embeddings salvos
with open("embeddings.json", "r") as f:
    stored_embeddings = json.load(f)

In [6]:
# Função para carregar e preprocessar um rosto detectado
def preprocess_face(image, box):
    x, y, width, height = box
    x, y = max(x, 0), max(y, 0)  # Evitar coordenadas negativas
    face = image[y:y+height, x:x+width]  # Recortar rosto
    face = cv2.resize(face, (160, 160))  # Redimensionar para o FaceNet
    face = np.array(face, dtype='float32') / 255.0  # Normalizar
    return face

In [7]:
# Função para comparar embeddings
def compare_embeddings(embedding1, embedding2):
    return 1 - cosine(embedding1, embedding2)  # Quanto mais próximo de 1, mais similar

In [8]:
# Função para reconhecer múltiplos rostos em uma imagem
def recognize_faces(image_path, threshold=0.5):
    image = cv2.imread(image_path)
    image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)  # Converter para RGB
    faces_detected = detector.detect_faces(image_rgb)  # Detectar rostos

    results = []

    for face in faces_detected:
        box = face["box"]
        processed_face = preprocess_face(image_rgb, box)
        processed_face = np.expand_dims(processed_face, axis=0)
        new_embedding = embedder.predict(processed_face)[0]  # Gerar embedding

        best_match = "Desconhecido"
        best_similarity = -1  

        # Comparar com os embeddings armazenados
        for label, embeddings in stored_embeddings.items():
            for stored_embedding in embeddings:
                similarity = compare_embeddings(new_embedding, stored_embedding)
                if similarity > best_similarity:  # Atualiza o melhor match
                    best_similarity = similarity
                    best_match = label

        if best_similarity >= threshold:
            results.append((best_match, box, best_similarity))
        else:
            results.append(("Desconhecido", box, best_similarity))

    return results, image

In [ ]:
# Caminho da imagem nova
new_image_path = "Reconhecimento Facial\\the-big-bang-theory-cast-together.jpg"

In [10]:
# Reconhecer os rostos
recognized_faces, image = recognize_faces(new_image_path)

In [11]:
# Exibir os resultados
for name, box, similarity in recognized_faces:
    x, y, w, h = box
    cv2.rectangle(image, (x, y), (x+w, y+h), (0, 255, 0), 2)  # Desenhar retângulo
    cv2.putText(image, f"{name} ({similarity:.2f})", (x, y-10), 
                cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)  # Nome + Similaridade

In [12]:
# Mostrar a imagem
cv2.imshow("Reconhecimento Facial", image)
k = cv2.waitKey(0)
if k == ord("s"): # se s for pressionado salva a imagem
    cv2.imwrite("resultado.jpg", image)
cv2.destroyAllWindows()